<a href="https://colab.research.google.com/github/PadmarajBhat/Real-Time-Analytics-on-Hadoop-Notes/blob/master/pyspark_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 46.5MB/s 
     |████████████████████████████████| 204kB 54.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=d19ea4d78286dfd5a6d21f24382726ab223989e11bdec4b3db13ad75aab35c28
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist



In [3]:
(train_x,train_y), (test_x, test_y) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
train_x.shape,train_y.shape, test_x.shape, test_y.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
pd_train= pd.DataFrame(train_x.reshape(60000,784))
pd_test = pd.DataFrame(test_x.reshape(10000,784))
pd_train.shape,pd_test.shape


((60000, 784), (10000, 784))

In [6]:
pd_train.values *.1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Note that we had to multiply .1 to convert the data to float else not accepted in the tensorflow.

In [7]:
model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(28,28))
    tf.keras.layers.Flatten()
    ,tf.keras.layers.Dense(256,activation="relu")
    ,tf.keras.layers.Dense(128,activation="relu")
    ,tf.keras.layers.Dense(10,activation="softmax")
])

model.compile(optimizer="Adam"
              , loss="sparse_categorical_crossentropy"
              , metrics=['accuracy']
             )

#model.fit(train,train_y, epochs=6)
model.fit(pd_train.values *.1,train_y,epochs=6)
model.evaluate(test_x,test_y)

W0901 07:41:24.395536 140368148768640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/6
60000/60000 [==============================] - 5s 86us/sample - loss: 0.2816 - acc: 0.9256
Epoch 2/6
60000/60000 [==============================] - 4s 72us/sample - loss: 0.1232 - acc: 0.9633
Epoch 3/6
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0966 - acc: 0.9715
Epoch 4/6
60000/60000 [==============================] - 4s 71us/sample - loss: 0.0807 - acc: 0.9756
Epoch 5/6
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0696 - acc: 0.9791
Epoch 6/6
10000/10000 [==============================] - 0s 40us/sample - loss: 1.1268 - acc: 0.9658


[1.1267635655719925, 0.9658]

Tensorflow is being fed with panda which indicates that entire data is loaded. This is not a good fit for Big Data case. We need the tensorflow to work on the RDD or Df rather than panda df. Hence spark read --> convert to panda --> tensorflow pipeline fails at the second stage. For this we have https://github.com/maxpumperla/elephas


Elephas: Take spark dataframe and trains it for the keras model. But the example does not talk about tensorflow keras but keras independent version. Not sure if tensorflow keras supports it.

Why we have to worry about tf.keras or keras as independent one ? 
  - tensorflow official supports keras and hence product would have enhancements
  - tensorflow has distributed stratergy to support distributed training. elephas has it too but may be tensorflow is better tested one becuase of larger community of dev and tester.
  - tensorflow has official distribution on hadoop and hence better than smaller community like elephas
  
  
There was mis understanding i guess, they dont provide keras but instead they transform the keras mode. Now it has to be tested so that if the tensorflow keras model is compatible with the elephas.


In [9]:
!pip install elephas

In [12]:
nb_classes = 10

from elephas.ml_model import ElephasEstimator
from keras import optimizers


adam = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(adam)

# Initialize SparkML Estimator and set all relevant properties
estimator = ElephasEstimator()
estimator.setFeaturesCol("scaled_features")             # These two come directly from pyspark,
estimator.setLabelCol("index_category")                 # hence the camel case. Sorry :)
estimator.set_keras_model_config(model.to_yaml())       # Provide serialized Keras model
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(1)  # We just use one worker here. Feel free to adapt it.
estimator.set_epochs(20) 
estimator.set_batch_size(128)
estimator.set_verbosity(1)
estimator.set_validation_split(0.15)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_8d9eee46c54f

In [13]:
estimator.fit(train_x,train_y)

ValueError: ignored